In [15]:
import numpy as np
import pandas as pd

from scipy.stats import wilcoxon

In [16]:
scenario = 'dtc'

In [17]:
data = pd.read_csv(f'evaluation_{scenario}.csv')

In [18]:
score = 'auc' if 'auc' in data.columns else 'r2'

In [19]:
data.head()

,Unnamed: 0,name,fold,auc,params,estimator,label
0,0,appendicitis,0,0.823529,"{'operator': '<=', 'random_state': 5, 'min_sam...",OperatorDecisionTreeClassifier,dtc_leq
1,1,appendicitis,1,0.786765,"{'operator': '<=', 'random_state': 5, 'min_sam...",OperatorDecisionTreeClassifier,dtc_leq
2,2,appendicitis,2,0.794118,"{'operator': '<=', 'random_state': 5, 'min_sam...",OperatorDecisionTreeClassifier,dtc_leq
3,3,appendicitis,3,0.772059,"{'operator': '<=', 'random_state': 5, 'min_sam...",OperatorDecisionTreeClassifier,dtc_leq
4,4,appendicitis,4,0.632353,"{'operator': '<=', 'random_state': 5, 'min_sam...",OperatorDecisionTreeClassifier,dtc_leq


In [20]:
def do_testing(pdf, metric):
    params = pdf['params'].drop_duplicates().values
    pdf0 = pdf[pdf['params'] == params[0]].sort_values('fold')
    pdf1 = pdf[pdf['params'] == params[1]].sort_values('fold')

    return pd.Series({
        'wilcoxon': wilcoxon(pdf0[metric], pdf1[metric], zero_method='zsplit').pvalue
    })

In [21]:
data['label'].drop_duplicates()

0           dtc_leq
20000         dtc_l
40000     adtc_full
60000     adtc_rand
80000          fdtc
100000         sdtc
Name: label, dtype: object

In [22]:
grouped = data\
    .groupby(['name', 'label'])\
    .apply(lambda pdf: pdf[score].values.tolist())\
    .reset_index(drop=False)\
    .rename(columns={0: score})

In [23]:
grouped

,name,label,auc
0,abalone9_18,adtc_full,"[0.788647342995169, 0.6467391304347826, 0.6865..."
1,abalone9_18,adtc_rand,"[0.788647342995169, 0.6467391304347826, 0.6865..."
2,abalone9_18,dtc_l,"[0.788647342995169, 0.6467391304347826, 0.6865..."
3,abalone9_18,dtc_leq,"[0.788647342995169, 0.6467391304347826, 0.6865..."
4,abalone9_18,fdtc,"[0.7910628019323671, 0.6467391304347826, 0.686..."
...,...,...,...
115,yeast-0-3-5-9_vs_7-8,adtc_rand,"[0.6983695652173914, 0.796153846153846, 0.6978..."
116,yeast-0-3-5-9_vs_7-8,dtc_l,"[0.6983695652173914, 0.796153846153846, 0.6978..."
117,yeast-0-3-5-9_vs_7-8,dtc_leq,"[0.6983695652173914, 0.796153846153846, 0.6978..."
118,yeast-0-3-5-9_vs_7-8,fdtc,"[0.696195652173913, 0.7999999999999999, 0.6879..."


In [44]:
def evaluate_one(grouped, pivot_postfix='leq'):
    pivot_row = grouped[grouped['label'] == f"{scenario}_{pivot_postfix}"].iloc[0]
    pivot_label = pivot_row['label']

    other_rows = grouped[grouped['label'] != pivot_label]

    result = {f'auc_{pivot_label}': np.mean(pivot_row[score])}

    for idx, row in other_rows.iterrows():
        result[f'auc_{row["label"]}'] = np.mean(row[score])
        result[f'p_{row["label"]}_l'] = wilcoxon(pivot_row[score], row[score], zero_method="zsplit", alternative="less").pvalue
        #result[f'p_{row["label"]}_g'] = wilcoxon(pivot_row[score], row[score], zero_method="zsplit", alternative="greater").pvalue

    return pd.Series(result)

In [45]:
def evaluate_all(data):
    return data.groupby("name").apply(evaluate_one)

In [46]:
evaluate_all(grouped)

,auc_dtc_leq,auc_adtc_full,p_adtc_full_l,auc_adtc_rand,p_adtc_rand_l,auc_dtc_l,p_dtc_l_l,auc_fdtc,p_fdtc_l,auc_sdtc,p_sdtc_l
name,,,,,,,,,,,
abalone9_18,0.761769,0.761769,5.000000e-01,0.761769,5.000000e-01,0.761769,5.000000e-01,0.761932,1.599006e-04,0.761769,5.000000e-01
appendicitis,0.774753,0.774753,5.000000e-01,0.774753,5.000000e-01,0.774753,5.000000e-01,0.774800,5.249264e-01,0.774753,5.000000e-01
bupa,0.692042,0.692249,8.100410e-01,0.692222,8.491318e-01,0.691741,9.725139e-01,0.692855,3.126914e-03,0.691812,9.175362e-01
cleveland-0_vs_4,0.888081,0.891356,1.320785e-01,0.891492,1.320785e-01,0.891727,1.320383e-01,0.891402,1.078043e-01,0.891727,1.320383e-01
ecoli1,0.954363,0.954363,5.000000e-01,0.954363,5.000000e-01,0.954363,5.000000e-01,0.955318,1.616892e-13,0.954363,5.000000e-01
glass0,0.832642,0.832642,5.000000e-01,0.832642,5.000000e-01,0.832642,5.000000e-01,0.833845,5.232257e-06,0.832642,5.000000e-01
haberman,0.656121,0.656287,6.025456e-02,0.656258,7.583640e-02,0.656338,6.205116e-02,0.656386,6.066495e-02,0.656338,6.205116e-02
hepatitis,0.746074,0.746297,3.124241e-01,0.746288,3.132339e-01,0.746375,3.749779e-01,0.749230,9.353505e-02,0.746062,6.457443e-01
lymphography,0.900070,0.903323,2.864238e-01,0.903323,2.864238e-01,0.903251,2.864238e-01,0.906929,8.088386e-03,0.903251,2.864238e-01
